## CNN Image Prediction

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root = './content/FashionMNIST',
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ]),
    download=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



In [11]:
class Network(nn.Module):

    def __init__(self):

        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [5]:
# setting off the PyTorch gradient calculation feature
# as we will not be calculating the gradients

# PyTorch automatically builds computation graph as out tensor
# flows through the network

torch.set_grad_enabled(False)

In [12]:
network = Network()

In [7]:
sample = next(iter(train_set))

In [8]:
image, label = sample
image.shape

torch.Size([1, 28, 28])

In [9]:
# batch_size = 1
image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [13]:
pred = network(image.unsqueeze(0))

In [14]:
pred.shape

torch.Size([1, 10])

In [15]:
pred

tensor([[-0.0971, -0.0451,  0.0156,  0.0175, -0.1073,  0.0373, -0.0091,  0.0807,  0.0463, -0.0691]])

In [16]:
label

9

In [17]:
pred.argmax(dim=1)

tensor([7])

In [19]:
F.softmax(pred, dim=1)

tensor([[0.0918, 0.0967, 0.1027, 0.1029, 0.0908, 0.1050, 0.1002, 0.1096, 0.1059, 0.0944]])

In [20]:
F.softmax(pred, dim=1).sum()

tensor(1.)